## How to add Pauses into the Blocklist
(this method does not simulate a true "pause" at the selected point: the toolhead will not necessarily come to a full stop)
1. First we need to define all input parameters for the simulation as usual

In [1]:
%matplotlib widget
from pyGCodeDecode import gcode_interpreter
from pyGCodeDecode import planner_block
from pyGCodeDecode import abaqus_file_generator
anisoprint_A4 = {
    #general properties
    "nozzle_diam"   :   0.4,
    "filament_diam" :   1.75,
    
    #default settings
    "velocity"      :   35,
    "acceleration"  :   1000,
    "jerk"          :   10,
    
    #axis max speeds
    "Vx"            :   180,
    "Vy"            :   180,
    "Vz"            :   30,
    "Ve"            :   33
    }

""" INITIAL POSITION:
    None        -> start from zero
    True        -> use first gcode G1 command as initial position
    [x,y,z,e]   -> non zero coordinates for initial position
"""
initial_position = None
pause_1_line = 685
pause_1_duration = 5
pause_2_line = 47601
pause_2_duration = 20
simulation = gcode_interpreter.simulate(filename=r"assembly_cake_20220831.gcode",printer=anisoprint_A4,initial_position=initial_position)

[##########] 100% of Planner Block Generation Done...
 >> pyGCodeDecode extracted 52503 states from assembly_cake_20220831.gcode and generated 52502 plannerblocks.
 
            Estimated time to travel all states with provided printer settings is 13452.796081720337 seconds.


In [2]:
abaqus_file_generator.generate_abaqus_events(simulation,output_filename=simulation.filename[:-6]+"-no_shift-abaqus.inp")

2. Then we want to split the blocklist in seperate parts to apply the timeshift method on certain blocks. Since we dont know beforehand which block in the blocklist will be at the required point in the GCODE, we can either search through the list for a certain GCODE line number or a certain height. (height might result in problems due to z-hop on previous layers)

3. lets use Line Numbering

In [3]:
blocklist = simulation.blocklist
blocklist_split0 = [block for block in blocklist if block.state_A.line_nmbr < pause_1_line]
blocklist_split1 = [block for block in blocklist if block.state_A.line_nmbr >= pause_1_line and block.state_A.line_nmbr < pause_2_line]
blocklist_split2 = [block for block in blocklist if block.state_A.line_nmbr > pause_2_line]

4. This is where we apply the timeshift

In [4]:
for block in blocklist_split1: block.timeshift(pause_1_duration)
for block in blocklist_split2: block.timeshift(pause_2_duration+pause_1_duration)

5. so now lets overwrite the existing blocklist of the simulation so we can use class functions such as plot_vel()

In [5]:
new_blocklist = list()
new_blocklist.extend(blocklist_split0)
new_blocklist.extend(blocklist_split1)
new_blocklist.extend(blocklist_split2)

simulation.blocklist = new_blocklist

let's see the changed timing in the velocity plot

In [7]:
simulation.print_summary(simulation.filename)

 >> pyGCodeDecode extracted 52503 states from assembly_cake_20220831.gcode and generated 52502 plannerblocks.
 
            Estimated time to travel all states with provided printer settings is 13477.796081720337 seconds.


6. Finally: let's generate the Abaqus INP file

In [8]:
abaqus_file_generator.generate_abaqus_events(simulation,output_filename=simulation.filename[:-6]+"-abaqus.inp")

# Done